In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from tqdm import tqdm
import pandas as pd
import itertools

In [2]:
file_path = 'LK_modified.xlsx'
all_sheets = pd.read_excel(file_path, sheet_name=None)
dfs = {sheet_name: pd.DataFrame(sheet_data) for sheet_name, sheet_data in all_sheets.items()}
df_first_sheet = dfs[list(dfs.keys())[0]]
df = dfs[list(dfs.keys())[1]]
df_second_sheet = dfs[list(dfs.keys())[2]]

In [3]:
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

In [4]:
rare_categories = df['content'].value_counts()[df['content'].value_counts() <= 1].index
rare_data = df[df['content'].isin(rare_categories)]

### Аугментация с помощью Т5 "cointegrated/rut5-base-paraphraser"

In [5]:
model_name = "cointegrated/rut5-base-paraphraser"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def generate_paraphrases(text, num_return_sequences=3, num_beams=5):
    input_text = "" + text + " </s>"
    encoding = tokenizer.encode_plus(input_text, padding=True, return_tensors="pt", max_length=128, truncation=True)
    input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=128,
        num_beams=num_beams,
        num_return_sequences=num_return_sequences,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True
    )

    return [tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True) for output in outputs]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
C:\Users\ivann\AppData\Roaming\Python\Python311\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
def augment_data(df, target_count=15):
    augmented_data = []
    category_counts = df['content'].value_counts()

    rare_categories = category_counts[category_counts < target_count].index

    for category in tqdm(rare_categories, desc="Augmenting categories"):
        category_df = df[df['content'] == category]
        num_current = len(category_df)
        num_additional = target_count - num_current

        row_cycle = itertools.cycle(category_df.iterrows())

        while num_additional > 0:
            index, row = next(row_cycle)
            paraphrases = generate_paraphrases(row['question'], num_return_sequences=min(num_additional, 5))

            for para in paraphrases:
                if num_additional <= 0:
                    break
                new_row = row.copy()
                new_row['question'] = para
                new_row['is_generated'] = True
                new_row['category'] = row['category']
                augmented_data.append(new_row)
                num_additional -= 1

    augmented_df = pd.DataFrame(augmented_data)
    df['is_generated'] = False
    return pd.concat([df, augmented_df], ignore_index=True)


In [7]:
print(df[df['content'] == rare_categories[48]]['question'])

101    Будет ли действовать электронная подпись, если...
Name: question, dtype: object


In [8]:
augment_data(df[df['content'] == rare_categories[48]], target_count=10)[['question', 'is_generated']]

Augmenting categories:   0%|          | 0/1 [00:00<?, ?it/s]C:\Users\ivann\AppData\Roaming\Python\Python311\site-packages\transformers\models\t5\tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
C:\Users\ivann\AppData\Roaming\Python\Python311\site-packages\transformers\models\t5\tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
Augmenting categories: 100%|██████████| 1/1 [00:06<00:00,  6.49s/it]
C:\Users\ivann\AppData\Local\Temp\ipykernel_3412\2338917637.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_gener

,question,is_generated
0,"Будет ли действовать электронная подпись, если...",False
1,"Действует ли электронная подпись, если персона...",True
2,Действует ли электронная подпись при неактуаль...,True
3,"Действует ли электронная подпись, если персона...",True
4,"Действует ли электронная подпись, если персона...",True
5,"Действует ли электронная подпись, если персона...",True
6,"Действует ли электронная подпись, если персона...",True
7,Действует ли электронная подпись при неактуаль...,True
8,"Действует ли электронная подпись, если персона...",True
9,"Действует ли электронная подпись, если персона...",True


- "Будет ли действовать электронная подпись, если персональные данные в учетной системе работодателя не актуальны?" - оригинал
- "Действует ли электронная подпись, если персональные данные в системе учетной записи работодателя не актуальны?"
- Действует ли электронная подпись при неактуальности персональных данных в системе учетной записи работодателя?
- "Действует ли электронная подпись, если персональные данные в системе учетной записи работодателя не будут актуальными?"
- "Действует ли электронная подпись, если персональные данные работодателя не будут актуальны?"
- "Действует ли электронная подпись, если персональные данные работодателя не будут актуальными?"
- "Действует ли электронная подпись, если персональные данные в системе учетной записи работодателя не актуальны?"
- Действует ли электронная подпись при неактуальности персональных данных в системе учетной записи работодателя?
- "Действует ли электронная подпись, если персональные данные в системе учетной записи работодателя не будут актуальными?"
- "Действует ли электронная подпись, если персональные данные работодателя не будут актуальны?"


In [9]:
print(df[df['content'] == rare_categories[12]]['question'])

1612    Как оформить доп. Дни отдыха по уходу за ребен...
Name: question, dtype: object


In [10]:
augment_data(df[df['content'] == rare_categories[12]], target_count=10)[['question', 'is_generated']]

Augmenting categories:   0%|          | 0/1 [00:00<?, ?it/s]C:\Users\ivann\AppData\Roaming\Python\Python311\site-packages\transformers\models\t5\tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
C:\Users\ivann\AppData\Roaming\Python\Python311\site-packages\transformers\models\t5\tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
Augmenting categories: 100%|██████████| 1/1 [00:03<00:00,  3.32s/it]
C:\Users\ivann\AppData\Local\Temp\ipykernel_3412\2338917637.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_gener

,question,is_generated
0,Как оформить доп. Дни отдыха по уходу за ребен...,False
1,Как оформить доп. Дни отдыха по уходу за ребен...,True
2,Как оформить доп. дни отдыха по уходу за ребен...,True
3,Как оформить доп. Дни отдыха по уходу за ребенком,True
4,Как оформить доп. Дни отдыха по уходу за детьм...,True
5,Как оформить доп. отдых по уходу за ребенком и...,True
6,Как оформить доп. Дни отдыха по уходу за ребен...,True
7,Как оформить доп. дни отдыха по уходу за ребен...,True
8,Как оформить доп. Дни отдыха по уходу за ребенком,True
9,Как оформить доп. Дни отдыха по уходу за детьм...,True


- Как оформить доп. Дни отдыха по уходу за ребенком инвалидом - оригинал
- Как оформить доп. Дни отдыха по уходу за ребенком инвалидом
- Как оформить доп. дни отдыха по уходу за ребенком инвалидом
- Как оформить доп. Дни отдыха по уходу за ребенком
- Как оформить доп. Дни отдыха по уходу за детьми инвалидами
- Как оформить доп. отдых по уходу за ребенком инвалидом
- Как оформить доп. Дни отдыха по уходу за ребенком инвалидом
- Как оформить доп. дни отдыха по уходу за ребенком инвалидом
- Как оформить доп. Дни отдыха по уходу за ребенком
- Как оформить доп. Дни отдыха по уходу за детьми инвалидами
